# Obtaining Daily Volatility 
Get hourly candlestick data from Binance Klines

### We calculate Realised Volatility for x days given a trading pair on Binance

In [48]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import concurrent.futures
import numpy as np
import time

def fetch_data(symbol, interval, start_time, end_time):
    url = "https://api.binance.com/api/v3/klines"
    params = {
            'symbol': symbol,
            'interval': interval,
            'startTime': start_time,
            'endTime': end_time,
            'limit': 1000
    }
    data = requests.get(url, params=params).json()
    return data

def threaded_fetching(symbol, interval, start_time, end_time):

    delta = 1000 * 60 * 60 * 24  # 1 day in milliseconds
    all_data = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        # distribute requests
        futures = [executor.submit(fetch_data, symbol, interval, t, min(t+delta, end_time)) for t in range(start_time, end_time, delta)]
        for future in concurrent.futures.as_completed(futures):
            all_data.extend(future.result())
    df = pd.DataFrame(all_data, columns=['open_time','open','high','low','close','volume','close_time','quote_asset_volume',
                                         'number_of_trades','taker_buy_base_asset_volume','taker_buy_quote_asset_volume','ignore'])
    df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
    return df

def calculate_volatility(df):
    df['close'] = pd.to_numeric(df['close'])
    df.set_index('open_time', inplace=True)
    df = df.resample('1H').last()
    df['log_return'] = np.log(df['close'] / df['close'].shift())
    df = df.resample('1D').std()
    df['log_return'] = df['log_return'] * np.sqrt(24) * 100  # annualize volatility
    return df[['log_return']]

def plot_volatility(df):
    df.plot()
    plt.title("Daily Realised Volatility")
    plt.xlabel("Date")
    plt.ylabel("Volatility (%)")
    plt.grid(True)
    plt.show()

def fetch_pair(symbol1, symbol2, interval, start_time, end_time):
    df1 = threaded_fetching(symbol=symbol1, interval=interval, start_time=start_time, end_time=end_time)
    df2 = threaded_fetching(symbol=symbol2, interval=interval, start_time=start_time, end_time=end_time)

    # Convert the 'close' column to numeric
    df1['close'] = pd.to_numeric(df1['close'])
    df2['close'] = pd.to_numeric(df2['close'])
    
    # Make sure that the two dataframes have the same index
    df1.set_index('open_time', inplace=True)
    df2.set_index('open_time', inplace=True)   
    
    # Resample to the millisecond level to maintain precision
    df1 = df1.resample('L').last().ffill()
    df2 = df2.resample('L').last().ffill()

    # Combine the two dataframes into one
    df = pd.concat([df1['open_time'],df1['close'], df2['close']], axis=1)
    df.columns = ['open_time',symbol1, symbol2]

    # Calculate the ratio of the two pairs
    df['close'] = df[symbol1] / df[symbol2]
    
    return df


### TODO: If trading pair not on Binance, write function to find volatility by routing through USDT

# Calculating LVR based on theoretical Formula

$$
\frac{l(\sigma^2,P)}{V(P)} = \frac{\frac{L\sigma^2}{4}\sqrt{P}}{L(2\sqrt{P} \  - \ P/\sqrt{P_b} \ - \ \sqrt{P_a})}
$$

$$
\frac{l(\sigma^2,P)}{V(P)} = \frac{\sigma^2\sqrt{P}}{4(2\sqrt{P} \  - \ P/\sqrt{P_b} \ - \ \sqrt{P_a})}
$$

In [49]:
import math

def calculate_ratio(sigma, P, Pb, Pa):
    numerator = (sigma ** 2) * math.sqrt(P)
    denominator = 4 * (2 * math.sqrt(P) - P / math.sqrt(Pb) - math.sqrt(Pa))
    ratio = numerator / denominator
    return ratio

#### Some helper functions to get Timestamps, token Prices, etc.

In [50]:
from web3 import Web3
import time
import requests

# Initialize a Web3 instance
w3 = Web3(Web3.HTTPProvider('https://eth.llamarpc.com'))

# get timestamp for a given block
def get_timestamp(block_number):
    block = w3.eth.get_block(block_number)
    return block.timestamp * 1000

# get timestamp for current time
def get_current_timestamp():
    return int(time.time()) * 1000

# get block from timestamp using llama fi endpoint https://coins.llama.fi/block/ethereum/{timestamp}

def get_block_from_timestamp(timestamp):
    url = f"https://coins.llama.fi/block/ethereum/{timestamp}"
    try:
        response = requests.get(url)
        data = response.json()
        block_number = data["height"]
        return block_number
    except requests.exceptions.RequestException as e:
        print("Error occurred:", e)
        return None
    
def get_date(timestamp):
    return datetime.fromtimestamp(timestamp/1000).strftime('%Y-%m-%d')   

def get_eth_price():
    url = "https://api.coingecko.com/api/v3/simple/price"
    params = {
        "ids": "ethereum",
        "vs_currencies": "usd"
    }

    try:
        response = requests.get(url, params=params)
        data = response.json()
        eth_price = data["ethereum"]["usd"]
        return eth_price
    except requests.exceptions.RequestException as e:
        print("Error occurred:", e)
        return None
    
def get_coin_historical(timestamp,address):
    url = f"https://coins.llama.fi/prices/historical/{timestamp}/ethereum:{address}?searchWidth=4h"
    try:
        response = requests.get(url)
        data = response.json()
        return data['coins'][f'ethereum:{address}']['price']
    except requests.exceptions.RequestException as e:
        print("Error occurred:", e)
        return None

#### Liquidity Math helper functions

In [51]:
import math

Q96 = 2**96

def get_tick_at_sqrt_ratio(sqrt_price_x96):
    tick = math.floor(math.log((sqrt_price_x96 / Q96) ** 2) / math.log(1.0001))
    return tick

def get_token_amounts(liquidity, sqrt_price_x96, tick_low, tick_high, token_0_decimal, token_1_decimal):
    sqrt_ratio_a = math.sqrt(1.0001 ** tick_low)
    sqrt_ratio_b = math.sqrt(1.0001 ** tick_high)
    
    current_tick = get_tick_at_sqrt_ratio(sqrt_price_x96)
    sqrt_price = sqrt_price_x96 / Q96
    
    amount_0_wei = 0
    amount_1_wei = 0
    if current_tick <= tick_low:
        amount_0_wei = math.floor(liquidity * ((sqrt_ratio_b-sqrt_ratio_a)/(sqrt_ratio_a*sqrt_ratio_b)))
    elif current_tick > tick_high:
        amount_1_wei = math.floor(liquidity * (sqrt_ratio_b-sqrt_ratio_a))
    elif current_tick >= tick_low and current_tick < tick_high: 
        amount_0_wei = math.floor(liquidity * ((sqrt_ratio_b-sqrt_price)/(sqrt_price*sqrt_ratio_b)))
        amount_1_wei = math.floor(liquidity * (sqrt_price-sqrt_ratio_a))
    
    amount_0_human = round(amount_0_wei / 10**token_0_decimal, token_0_decimal)
    amount_1_human = round(amount_1_wei / 10**token_1_decimal, token_1_decimal)
    return [amount_0_human, amount_1_human]

def priceToSqrtPrice(price,decimal0, decimal1):
    # convert this to an equation, multiply the square root of price multiplied by 10 to the power of decimal 1 - 0 with 2 to the power of 96
    sqrtPrice = math.sqrt(price * 10 ** (decimal1 - decimal0)) * 2 ** 96
    return int(sqrtPrice)

### Fetch On-Chain Data for Uniswap V3 NFT Positions

In [52]:
from abi import v3_pool_abi, erc20_abi, uniswap_positions_nft_abi
import pandas as pd
import concurrent.futures

def get_position_data_block_range(block_range, position_id, pool_address, token0_decimals, token1_decimals):
    data_list = []
    for block in block_range:
        try:
            position_data = get_position_nft_data(position_id, block)
        except:
            print(f"TokenId not found for block {block}")
            # move on to the next block
            continue
        
        tickLower = position_data[5]
        tickUpper = position_data[6]
        tokensOwed0 = position_data[10]
        tokensOwed1 = position_data[11]
        liquidity = position_data[7]

        pool_contract = w3.eth.contract(address=pool_address, abi=v3_pool_abi)
        current_tick = pool_contract.functions.slot0().call(block_identifier=block)[1]
        upper_price = (1.0001 ** tickUpper) / (10 ** (token1_decimals - token0_decimals))
        lower_price = (1.0001 ** tickLower) / (10 ** (token1_decimals - token0_decimals))
        current_price = (1.0001 ** current_tick) / (10 ** (token1_decimals - token0_decimals))

        sqrtPriceLower = priceToSqrtPrice(lower_price, token0_decimals, token1_decimals)
        sqrtPriceUpper = priceToSqrtPrice(upper_price, token0_decimals, token1_decimals)
        sqrtPriceCurrent = priceToSqrtPrice(current_price, token0_decimals, token1_decimals)

        amounts = get_token_amounts(liquidity, sqrtPriceCurrent, tickLower, tickUpper, token0_decimals, token1_decimals)
        amount0 = amounts[0]
        amount1 = amounts[1]

        pool_value = (amount0 * current_price) + amount1
        timestamp = get_timestamp(block)
        data_list.append({
            'block': block,
            'current_tick': current_tick,
            'upper_price': upper_price,
            'lower_price': lower_price,
            'current_price': current_price,
            'amount0': amount0,
            'amount1': amount1,
            'timestamp': timestamp,
            'date': get_date(timestamp),
            'pool_value': pool_value,
            'token0_usd': get_coin_historical(timestamp/1000,token0),
            'token1_usd': get_coin_historical(timestamp/1000,token1), 
        })

    return data_list

def chunks(lst, n):
    # Yield successive n-sized chunks from lst
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def get_position_details(position_id, pool_address, start_timestamp, end_timestamp, num_threads=7):

    start_block = get_block_from_timestamp(start_timestamp / 1000)
    end_block = w3.eth.block_number

    # Get the token0 and token1 contract details
    position_data = get_position_nft_data(position_id, end_block)
    global token0, token1
    token0 = position_data[2]
    token1 = position_data[3]

    token0_contract = w3.eth.contract(address=token0, abi=erc20_abi)
    token1_contract = w3.eth.contract(address=token1, abi=erc20_abi)
    token0_decimals = token0_contract.functions.decimals().call(block_identifier=end_block)
    token1_decimals = token1_contract.functions.decimals().call(block_identifier=end_block)

    block_step = 7200  # number of blocks in one day
    blocks = list(range(start_block, end_block, block_step))
    block_chunks = list(chunks(blocks, len(blocks) // num_threads))

    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        results = list(executor.map(get_position_data_block_range, block_chunks,
          [position_id]*len(block_chunks), [pool_address]*len(block_chunks), 
          [token0_decimals]*len(block_chunks), [token1_decimals]*len(block_chunks)))

    details = [detail for sublist in results for detail in sublist]  # flatten the results

    df = pd.DataFrame(details)
    return df

### Fetch On-Chain Dat for Range Protocol Vaults

In [53]:
import pandas as pd
import concurrent.futures
from abi import pool_abi, v3_pool_abi, erc20_abi

def get_data_for_block_range(block_range):
    details = []
    for block in block_range:
        upper_tick = pool_contract.functions.upperTick().call(block_identifier=block)
        lower_tick = pool_contract.functions.lowerTick().call(block_identifier=block)
        fees = pool_contract.functions.getCurrentFees().call(block_identifier=block)
        fee0 = fees[0] / 10 ** token0_decimals
        fee1 = fees[1] / 10 ** token1_decimals
        underlying_balances = pool_contract.functions.getUnderlyingBalances().call(block_identifier=block)
        amount0 = underlying_balances[0] / 10 ** token0_decimals
        amount1 = underlying_balances[1] / 10 ** token1_decimals
        current_tick = v3_pool_contract.functions.slot0().call(block_identifier=block)[1]
        upper_price = (1.0001 ** upper_tick) / (10**(token1_decimals-token0_decimals))
        lower_price = (1.0001 ** lower_tick) / (10**(token1_decimals-token0_decimals))
        current_price = (1.0001 ** current_tick) / (10**(token1_decimals-token0_decimals))
        pool_value = (amount0 * current_price) + amount1
        current_fees = (fee0 * current_price) + fee1
        timestamp = get_timestamp(block)
        details.append({
            'block': block,
            'lower_price': lower_price, 
            'upper_price': upper_price,
            'current_price': current_price, 
            'pool_value': pool_value,
            'current_fees': current_fees,
            'timestamp': timestamp,
            'date': get_date(timestamp),
            'token0_usd': get_coin_historical(timestamp/1000,token0),
            'token1_usd': get_coin_historical(timestamp/1000,token1),
        })
    return details

def chunks(lst, n):
    # Yield successive n-sized chunks from lst
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def get_range_position_details(pool_address, start_timestamp, end_timestamp, num_threads=7):

    # Initialize a Web3 instance
    w3 = Web3(Web3.HTTPProvider('https://eth-mainnet.g.alchemy.com/v2/OV2Uqs1tC70MAtl88BcFArffiXobkx6T'))

    # Assume a function get_block_number(timestamp)
    start_block = get_block_from_timestamp(start_timestamp/1000)
    end_block = get_block_from_timestamp(end_timestamp/1000)

    global pool_contract
    pool_contract = w3.eth.contract(address=pool_address, abi=pool_abi)
    name = pool_contract.functions.name().call(block_identifier=start_block)
    global token0
    token0 = pool_contract.functions.token0().call(block_identifier=start_block)
    global token1
    token1 = pool_contract.functions.token1().call(block_identifier=start_block)

    token0_contract = w3.eth.contract(address=token0, abi=erc20_abi)
    token1_contract = w3.eth.contract(address=token1, abi=erc20_abi)
    global token0_decimals
    token0_decimals = token0_contract.functions.decimals().call(block_identifier=start_block)
    global token1_decimals
    token1_decimals = token1_contract.functions.decimals().call(block_identifier=start_block)
    token0_name = token0_contract.functions.name().call(block_identifier=start_block)
    token1_name = token1_contract.functions.name().call(block_identifier=start_block)

    # Get Pool in Uniswap V3
    pool = pool_contract.functions.pool().call(block_identifier=start_block)
    global v3_pool_contract
    v3_pool_contract = w3.eth.contract(address=pool, abi=v3_pool_abi)
    block_step = 7200 # number of blocks in one 1 day
    blocks = list(range(start_block, end_block, block_step))
    block_chunks = list(chunks(blocks, len(blocks)//num_threads))

    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        results = list(executor.map(get_data_for_block_range, block_chunks))

    details = [detail for sublist in results for detail in sublist]  # flatten the results

    # Convert to DataFrame
    df = pd.DataFrame(details)

    return df,token0,token1

### Helper Functions for Getting Volatility Data and Calculating LVR

In [54]:
def get_volatility(symbol):
    start = get_timestamp(17285193)
    end = get_current_timestamp()
    df = threaded_fetching(symbol, '1m', start, end)
    df2 = df.copy()
    volatility_df = calculate_volatility(df)
    volatility_df.reset_index(inplace=True)

    return volatility_df

# ethusdc_volatility_df = get_volatility('ETHUSDC')
# ethusdc_volatility_df.to_csv('ethusdc_volatility.csv', index=False)

def get_volatility_routed(AssetA,AssetB,RouteAsset):
    # Fetch data for LQTYUSDT and ETHUSDT
    df1 = threaded_fetching(f'{AssetA}{RouteAsset}', '1m', start, end)
    df2 = threaded_fetching(f'{AssetB}{RouteAsset}', '1m', start, end)

    # Merge df1 and df2 on open_time, for open, high, low and close, divide df1 value by df2 value
    df1[['open', 'high', 'low', 'close']] = df1[['open', 'high', 'low', 'close']].astype('float64')
    df2[['open', 'high', 'low', 'close']] = df2[['open', 'high', 'low', 'close']].astype('float64')
    df1[['open', 'high', 'low', 'close']] = df1[['open', 'high', 'low', 'close']] / df2[['open', 'high', 'low', 'close']]

    # Calculate volatility and reset index
    volatility_df = calculate_volatility(df1)
    volatility_df.reset_index(inplace=True)

    return volatility_df

# lqtyeth_volatility_df = get_volatility_routed('LQTY','ETH','USDT')
# lqtyeth_volatility_df.to_csv('lqtyeth_volatility.csv', index=False)

def calculate_LVR(volatility_df, nft_position):
    # Convert open_time and date columns to datetime format
    volatility_df['open_time'] = pd.to_datetime(volatility_df['open_time'])
    nft_position['date'] = pd.to_datetime(nft_position['date'])

    # Merge volatility_df and nft_position based on open_time and date
    merged_df = pd.merge(volatility_df, nft_position, how='inner', left_on='open_time', right_on='date')

    # Compute daily LVR ratio by using calculate_ratio function
    merged_df['daily_LVR_ratio'] = merged_df.apply(lambda x: calculate_ratio(x['log_return'], x['current_price'], x['upper_price'], x['lower_price']), axis=1)

    # If upper_price and lower_price are the same, then the daily LVR ratio is 0
    merged_df.loc[merged_df['upper_price'] == merged_df['lower_price'], 'daily_LVR_ratio'] = 0

    # If current_price is outside of upper_price and lower_price, then the daily LVR ratio is 0
    merged_df.loc[(merged_df['current_price'] > merged_df['upper_price']) | (merged_df['current_price'] < merged_df['lower_price']), 'daily_LVR_ratio'] = 0

    # If current_price is outside of upper_price and lower_price, create a column called in_range and set it to False
    merged_df['in_range'] = merged_df.apply(lambda x: True if (x['current_price'] <= x['upper_price']) & (x['current_price'] >= x['lower_price']) else False, axis=1)

    # Compute daily LVR by dividing daily LVR ratio by 10000 and multiplying with Pool Value
    merged_df['daily_LVR'] = merged_df.apply(lambda x: x['daily_LVR_ratio']/10000 * x['pool_value'], axis=1)

    # Create column for cumulative LVR  
    merged_df['cumulative_LVR'] = merged_df['daily_LVR'].cumsum()


    return merged_df

def calculate_LVR_range(volatility_df, range_position):
    # convert open_time in volatility_df and date in range_position to the same type
    volatility_df['open_time'] = pd.to_datetime(volatility_df['open_time'])
    range_position['date'] = pd.to_datetime(range_position['date'])

    # merge volatility_df and range_position based on open_time and date
    merged_df = pd.merge(volatility_df, range_position, how='inner', left_on='open_time', right_on='date')

    # compute daily LVR ratio by using calculate ratio
    merged_df['daily_LVR_ratio'] = merged_df.apply(lambda x: calculate_ratio(x['log_return'], x['current_price'], x['upper_price'], x['lower_price']), axis=1)

    # if upper_price and lower_price are the same, then the daily LVR ratio is 0
    merged_df.loc[merged_df['upper_price'] == merged_df['lower_price'], 'daily_LVR_ratio'] = 0

    # if current_price is outside of upper_price and lower_price, then the daily LVR ratio is 0
    merged_df.loc[(merged_df['current_price'] > merged_df['upper_price']) | (merged_df['current_price'] < merged_df['lower_price']), 'daily_LVR_ratio'] = 0

    # if current_price is outside of upper_price and lower_price, create a column called in_range and set it to False
    merged_df['in_range'] = merged_df.apply(lambda x: True if (x['current_price'] <= x['upper_price']) & (x['current_price'] >= x['lower_price']) else False, axis=1)

    # compute daily LVR by dividing daily LVR ratio by 10000 and multiplying with Pool Value
    merged_df['daily_LVR'] = merged_df.apply(lambda x: x['daily_LVR_ratio']/10000 * x['pool_value'], axis=1)

    # create column for cumulative LVR   
    merged_df['cumulative_LVR'] = merged_df['daily_LVR'].cumsum()

    # create column for collected_fees
    previous_fee = merged_df['current_fees'].iloc[0]
    fee_to_add = 0

    merged_df['collected_fees'] = merged_df['current_fees']

    # Iterate over the dataframe
    for i, current_fee in enumerate(merged_df['current_fees']):

        if current_fee < (previous_fee * 0.1):
            fee_to_add = previous_fee
            merged_df['collected_fees'].iloc[i] += fee_to_add
        else:
            merged_df['collected_fees'].iloc[i] += fee_to_add

        previous_fee = merged_df['collected_fees'].iloc[i]
    
    return merged_df


### Preparing Data for NFT Positions

In [58]:
TOP_LQTY_WETH_POSITIONS = [366096, 506262, 453456]
TOP_USDC_WETH_POSITIONS = [250274, 520645, 515539]
start = get_timestamp(17285193)
end = get_current_timestamp()
print(start)
print(end)

LQTY_WETH = "0xD1D5A4c0eA98971894772Dcd6D2f1dc71083C44E"
USDC_WETH = "0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640"

# get volatility data from csv
lqtyeth_volatility_df = get_volatility_routed('LQTY','ETH','USDT')
ethusdc_volatility_df = get_volatility('ETHUSDC')

for position in TOP_LQTY_WETH_POSITIONS:
    nft_position = get_position_details(position, LQTY_WETH,start,end, num_threads=7)
    nft_position = calculate_LVR(lqtyeth_volatility_df,nft_position)
    nft_position.to_csv(f"lqty_weth_{position}.csv", index=False)

for position in TOP_USDC_WETH_POSITIONS:
    nft_position = get_position_details(position, USDC_WETH,start,end, num_threads=7)
    nft_position = calculate_LVR(ethusdc_volatility_df,nft_position)
    nft_position.to_csv(f"usdc_weth_{position}.csv", index=False)

1684397831000
1690275327000
TokenId not found for block 17414793
TokenId not found for block 17285193
TokenId not found for block 17349993
TokenId not found for block 17292393
TokenId not found for block 17421993
TokenId not found for block 17357193
TokenId not found for block 17299593
TokenId not found for block 17364393
TokenId not found for block 17371593
TokenId not found for block 17306793
TokenId not found for block 17313993TokenId not found for block 17378793

TokenId not found for block 17321193
TokenId not found for block 17385993
TokenId not found for block 17328393
TokenId not found for block 17393193
TokenId not found for block 17335593
TokenId not found for block 17400393
TokenId not found for block 17407593
TokenId not found for block 17342793
TokenId not found for block 17285193
TokenId not found for block 17349993
TokenId not found for block 17292393
TokenId not found for block 17357193
TokenId not found for block 17299593
TokenId not found for block 17364393
TokenId no

### Preparing Data for Range Positions

In [60]:
start = get_timestamp(17285193)
end = get_current_timestamp()

# get volatility data from csv
lqtyeth_volatility_df = get_volatility_routed('LQTY','ETH','USDT')
ethusdc_volatility_df = get_volatility('ETHUSDC')

lqtyeth_pool = "0x350D81A7733Ee6b001966e0844A0ebb096FAbF0f"
range_position,token0,token1 = get_range_position_details(lqtyeth_pool, start, end)
range_position = calculate_LVR_range(lqtyeth_volatility_df, range_position)
range_position.to_csv('lqtyeth_range_position.csv', index=False)

usdceth_pool = "0x9Ad8d0df2dA118DcE898b7F5BD9Ab749c593A5d9"
range_position,token0,token1 = get_range_position_details(usdceth_pool, start, end)
range_position = calculate_LVR_range(ethusdc_volatility_df, range_position)
range_position.to_csv('usdceth_range_position.csv', index=False)

/var/folders/lv/l4xn1q3s7jxgwb_811k8vqrh0000gn/T/ipykernel_10119/3514106941.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['collected_fees'].iloc[i] += fee_to_add
/var/folders/lv/l4xn1q3s7jxgwb_811k8vqrh0000gn/T/ipykernel_10119/3514106941.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['collected_fees'].iloc[i] += fee_to_add
/var/folders/lv/l4xn1q3s7jxgwb_811k8vqrh0000gn/T/ipykernel_10119/3514106941.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu